In [1]:
import calliope
import pandas as pd



def get_tidy_data(model: calliope.Model, coefficient: str) -> pd.DataFrame:
    r"""
    Get data from a model and format it for plotting.
    """
    data = (
        model.get_formatted_array(coefficient, index_format="multiindex")
        .to_dataframe()
        .reset_index()
    )

    return data


In [2]:
path_inputs = "../run-prebuilt-sector-coupled-euro-calliope/build/eurospores/inputs/2016_res_12h.nc"
model = calliope.read_netcdf(path_inputs)
# sorted([loctec for loctec in model.inputs.loc_techs.to_pandas().to_list() if "awe" in loctec])

FileNotFoundError: [Errno 2] No such file or directory: b'/home/jlauner/repos/jw4a-d2.1/run-prebuilt-sector-coupled-euro-calliope/build/eurospores/inputs/2016_res_12h.nc'

In [4]:
path_results = "../run-prebuilt-sector-coupled-euro-calliope/build/eurospores/outputs/2016_res_6h_noveltech.nc"
# "../../sector-coupled-euro-calliope-pre-built-internal/build/national/outputs/2016_res_12h.nc"

model = calliope.read_netcdf(path_results)

# boundaries = gpd.load_file("../data/europe-98-zones.geojson/europe-98-zones.geojson")

In [9]:
novel_techs = [i for i in sorted(list(model.inputs.techs.values)) if "awe" in i or "floating" in i ]
print("novel techs in model.inputs.techs: ", novel_techs)
df = get_tidy_data(model, "cost_energy_cap")
print("novel techs in model results cost_energy_cap: ", df.loc[df.techs.isin(novel_techs)].techs.unique())

novel techs in model.inputs.techs:  ['awe_deep_fw1', 'awe_onshore_sw', 'awe_shallow_fw1']
novel techs in model results cost_energy_cap:  ['awe_shallow_fw1' 'awe_deep_fw1' 'awe_onshore_sw']


cost_energy_cap
costs    locs   techs                                                   
monetary SWE_4  biofuel_to_diesel                              93.000000
         ITA_5  hydrogen_to_liquids                           150.000000
         FRA_6  chp_hydrogen                                   80.000000
         ITA_4  biofuel_to_methane                            214.285714
         FRA_6  hydrogen_to_methane                            30.000000
...                                                                  ...
         LVA_1  dac                                            15.920000
         ESP_10 chp_methane_extraction_heat_storage_big         0.000000
         ESP_1  hydrogen_to_methanol                          150.000000
         ITA_2  hydrogen_to_methane                            30.000000
         DNK_2  dac                                            15.920000

[4141 rows x 1 columns]

In [4]:
# look at the values of cost
df.loc[df.techs.isin(novel_techs)]

,costs,locs,techs,cost_energy_cap
21,monetary,CYP_1,awe_onshore_sw,60.87
46,monetary,POL_3,awe_onshore_sw,60.87
66,monetary,ITA_4,awe_onshore_sw,60.87
132,monetary,NLD_1,awe_deep_fw1,169.66
136,monetary,DEU_6,awe_onshore_sw,60.87
...,...,...,...,...
4033,monetary,SVN_1,awe_deep_fw1,169.66
4034,monetary,DEU_3,awe_onshore_sw,60.87
4044,monetary,ALB_1,awe_shallow_fw1,153.35
4087,monetary,ESP_2,awe_onshore_sw,60.87


In [18]:
import numpy as np
df = get_tidy_data(model, "energy_cap_max")
cap_wind_offshore = df.loc[df.techs.isin(["wind_offshore"])]
cap_wind_offshore.loc[~cap_wind_offshore.energy_cap_max.isin([np.inf, -np.inf])]

,locs,techs,energy_cap_max
36,ROU_2,wind_offshore,0.000000
96,CHE_1,wind_offshore,0.000000
307,ROU_3,wind_offshore,1.337488
534,POL_5,wind_offshore,1.727793
547,ESP_11,wind_offshore,0.400497
...,...,...,...
7482,FRA_4,wind_offshore,0.691803
7496,ESP_10,wind_offshore,0.148821
7727,FRA_11,wind_offshore,0.000000
7770,NOR_4,wind_offshore,0.144248


In [8]:
model_data = model._model_data.cost_energy_cap.to_pandas().T
model_data.loc[model_data.index.map(lambda x: any([nt in x for nt in novel_techs]))]

costs,monetary
loc_techs_investment_cost,
CYP_1::awe_onshore_sw,60.87
POL_3::awe_onshore_sw,60.87
ITA_4::awe_onshore_sw,60.87
NLD_1::awe_deep_fw1,169.66
DEU_6::awe_onshore_sw,60.87
...,...
SVN_1::awe_deep_fw1,169.66
DEU_3::awe_onshore_sw,60.87
ALB_1::awe_shallow_fw1,153.35


In [9]:
energy_cap = get_tidy_data(model, "energy_cap").sort_values(by=["locs", "techs"])
df = energy_cap.loc[energy_cap.techs.isin(["wind_offshore", "wind_onshore_monopoly", "wind_onshore_competing"])].round(2)

In [23]:
df.set_index(["locs", "techs"]).to_csv("national_energy_cap.csv")

In [25]:
model.run(force_rerun=True, build_only=True)

    (type=<class 'pyomo.core.base.param.IndexedParam'>) on block unknown with
    a new Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [ ]:
[getattr(model._backend_model, d).pprint() for d in model._backend_model.__dir__() if "area" in d]
# model._backend_model.group_constraint_loc_techs_wind_offshore_shallow_cap_max_DEU_1.pprint()

In [49]:
with open("model.txt", "w") as f:
    for d in model._backend_model.__dir__():
        if "cost" in d:
            getattr(model._backend_model, d).pprint(f)


            
